In [139]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime
from meteostat import Stations, Daily, Point

In [148]:
flights = pd.read_csv('../data/flights_train.csv', 
                      parse_dates = ['flight_date']).reset_index(drop=True)

url="https://raw.githubusercontent.com/ravisurdhar/flight_delays/master/airports.csv"
us_airports=pd.read_csv(url)

In [149]:
airports = np.unique((np.append(flights['from'].unique(), flights['to'].unique())))
airport_localisation = us_airports[us_airports['IATA_CODE'].isin(airports)]

In [150]:
lats = airport_localisation['LATITUDE'].tolist()
longs = airport_localisation['LONGITUDE'].tolist()
iatas = airports.tolist()
list_geo = list(zip(lats,longs))
iatas = airports.tolist()
list_data=[]

for i in range(len(list_geo)):
    point = Point(list_geo[i][0], list_geo[i][1])
    start = datetime(2011, 9, 1)
    end = datetime(2012, 11, 14)
    data = Daily(point, start, end)
    df = data.fetch()
    df = df.drop(labels=['snow', 'wdir', 'wpgt', 'pres', 'tsun'], axis=1)
    list_data.append(df.reset_index())

weather_data = pd.concat(list_data, axis=0)

In [151]:
iatas_repeated = [iata for iata in iatas for i in range(list_data[0].shape[0])]
weather_data['airport_code'] = iatas_repeated

In [147]:
weather_data.to_csv('../data/weather_data.csv')